# Building an ETL Pipeline for Fall Seasonal Bakery Items from Walmart

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import re

In [3]:
url = 'https://www.walmart.com/browse/food/seasonal-bakery/976759_976779_7443156_4622028?povid=976759_ItemCarousel_4302028_Fallbakerytreats_ViewAll_Rweb_Sept_05'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

### Web Scraping

In [4]:
r = requests.get(url, headers=headers)

In [5]:
# Check for successful response
if r.status_code == 403:
    print("Access Forbidden. The server may be blocking requests.")
else:
    r = r.text

In [ ]:
# Parse HTML and make readable
soup = BeautifulSoup(r, 'html.parser')
print(soup.prettify())

In [7]:
# Obtain prices
prices = soup.find_all('span', class_='w_iUH7', string=lambda t: t and 'price' in t.lower())
cur_prices = [str(i).split('$')[1].split('<')[0] for i in prices]

# Obtain titles
title_start = soup.find_all('span', class_='normal dark-gray mb0 mt1 lh-title f6 f5-l lh-copy')
title = [i.get_text().split(',')[0] for i in title_start]

# Get weights from ounces in title
weights = [i.get_text().split(',')[1] for i in title_start]

# Get shelf stable status
shelf_stable = [bool(re.search('shelf stable', i.get_text().lower())) for i in title_start]

# Get current ratings
rating_start = soup.find_all('span', class_='w_iUH7', string=lambda b: b and 'stars' in b.lower())
cur_rating = [float(i.get_text().split(' ')[0]) for i in rating_start]

In [8]:
print(cur_prices)
print(title)
print(weights)
print(shelf_stable)
print(cur_rating)

['4.48', '3.98', '5.84', '4.48', '6.47', '7.47', '5.84', '3.98', '3.98']
['Freshness Guaranteed Tri Pumpkin Face Sugar Cookies', 'Freshness Guaranteed Harvest Orange Frosted Sugar Cookies', 'Freshness Guaranteed Pumpkin Cake Roll', 'Freshness Guaranteed Pumpkin Shaped Vanilla Frosted Cookie', 'Freshness Guaranteed Assorted Harvest Cookies', 'Freshness Guaranteed Sugared Shortbread Cookies', 'Freshness Guaranteed Frosted Sugar Cookies', 'Walmart Freshness Guaranteed Fall Vanilla & Chocolate Mini Cupcakes', 'Freshness Guaranteed Half Moon Iced Fall Bakery Cookies']
[' Baked', ' 13.5 oz', ' 18 oz', ' 13.5 OZ', ' 21 oz', ' 16 oz', ' Harvest Yellow and Orange 24.3 oz', ' 10 oz', ' 10oz']
[False, False, False, True, False, False, False, False, False]
[]


Only one rating was found for the items. We will use the JSON response to obtain rating data.

In [ ]:
# Get JSON data from script tag
script_tag = soup.find('script', id='__NEXT_DATA__')

if script_tag:
    json_content = script_tag.string  # Get the content of the script tag

    json_data = json.loads(json_content)
    
    print(json_data)
else:
    print("JSON data not found in the HTML.")

#### Exploring the Data

In [10]:
type(json_data)

dict

A custom function will be defined in utils.py to understand the structure of this JSON data.

In [11]:
from utils import print_dict_keys

In [ ]:
print_dict_keys(json_data)

In [ ]:
# Index into individual items
json_data['props']['pageProps']['initialData']['searchResult']['itemStacks'][0]['items']

In [14]:
fall_items = json_data['props']['pageProps']['initialData']['searchResult']['itemStacks'][0]['items']
len(fall_items)

37

Now there are 37 items in the list from the JSON response. Much better than the 9 from parsing the HTML. The dict contains fields such as priceInfo, name, shortDescription among other useful fields.

#### Extracting Price

In [15]:
prices = [item.get('priceInfo', {}).get('linePrice', 'Price not available') for item in fall_items]

In [16]:
prices[:10]

['$4.48',
 '$3.98',
 '$5.84',
 '$4.48',
 '$6.47',
 '$7.47',
 '$5.84',
 '$3.98',
 'Price not available',
 '$3.98']

In [17]:
# Conver strings to floats
def safe_convert_price(price_str):
    try:
        return float(price_str.replace('$', '').replace(',', ''))
    except ValueError:
        return None

In [18]:
new_prices = [safe_convert_price(i) for i in prices]

In [19]:
new_prices[:10]

[4.48, 3.98, 5.84, 4.48, 6.47, 7.47, 5.84, 3.98, None, 3.98]

Float conversion has worked. Now we will collect the title, description, seller name, rating and number of reviews.

#### Extracting Title

In [20]:
name = [item.get('name', None) for item in fall_items]

In [21]:
name[:10]

['Freshness Guaranteed Tri Pumpkin Face Sugar Cookies, Baked, 11.2 oz, 9 Count',
 'Freshness Guaranteed Harvest Orange Frosted Sugar Cookies, 13.5 oz, 10 Count, Round, Shelf-Stable/Ambient, and Ready to Eat',
 'Freshness Guaranteed Pumpkin Cake Roll, 18 oz, 1 Count',
 'Freshness Guaranteed Pumpkin Shaped Vanilla Frosted Cookie, 13.5 OZ, 10 Count, Soft and Chewy Prepared Dessert, Shelf Stable',
 'Freshness Guaranteed Assorted Harvest Cookies, 21 oz, 45 Count',
 'Freshness Guaranteed Sugared Shortbread Cookies, 16 oz, 18 Count, Baked',
 'Freshness Guaranteed Frosted Sugar Cookies, Harvest Yellow and Orange 24.3 oz, 18 Count',
 'Walmart Freshness Guaranteed Fall Vanilla & Chocolate Mini Cupcakes, 10 oz, 12 Count',
 None,
 'Freshness Guaranteed Half Moon Iced Fall Bakery Cookies, 10oz, 10 Count, Ready to Eat']

#### Extracting Description

In [22]:
short_desc = [item.get('shortDescription', None) for item in fall_items]

In [23]:
short_desc[:10]

['<li>Freshness Guaranteed Tri Pumpkin Face Sugar Cookies</li><li>Kosher Dairy (OU-D)</li><li>A soft, multi-colored jack-o-lantern sugar cookie</li><li>Cookie shape: pumpkin</li>',
 '<li>Sugar cookies topped with creamy orange frosting</li><li>Dotted with fall inspired sprinkles to welcome autumn</li><li>Conveniently packaged for easy snacking on the go</li>',
 'Pumpkin Cake Roll, Kosher certified, contains no high fructose corn syrup',
 '<li>Freshness Guaranteed Pumpkin Shaped Vanilla Frosted Cookie, 13.5 OZ, 10 Count</li><li>Soft and Chewy Prepared Dessert</li><li>Topped with a smooth, creamy vanilla frosting and Pumpkin design</li>',
 '<li>Includes about 45 cookies</li><li>Flavors include chocolate cookies, chocolate chip cookies, cinnamon spice cookies, shortbread cookies</li><li>Some cookies are topped with orange and yellow</li>',
 '<li>Freshness Guaranteed Sugared Shortbread Cookies, 16 oz, 18 Count</li><li>Shortbread cookies topped with a sugared coating</li><li>Nut-Free</li>',

In [24]:
# Function to remove li tags from description
def replace_li(desc):
    # Ensure desc is a string before replacing
    if isinstance(desc, str):
        new_desc = desc.replace('<li>', ' ').replace('</li>', ' ')
        return new_desc.strip()
    else:
        return None

In [25]:
new_desc = [replace_li(i) for i in short_desc]

In [26]:
new_desc[:10]

['Freshness Guaranteed Tri Pumpkin Face Sugar Cookies  Kosher Dairy (OU-D)  A soft, multi-colored jack-o-lantern sugar cookie  Cookie shape: pumpkin',
 'Sugar cookies topped with creamy orange frosting  Dotted with fall inspired sprinkles to welcome autumn  Conveniently packaged for easy snacking on the go',
 'Pumpkin Cake Roll, Kosher certified, contains no high fructose corn syrup',
 'Freshness Guaranteed Pumpkin Shaped Vanilla Frosted Cookie, 13.5 OZ, 10 Count  Soft and Chewy Prepared Dessert  Topped with a smooth, creamy vanilla frosting and Pumpkin design',
 'Includes about 45 cookies  Flavors include chocolate cookies, chocolate chip cookies, cinnamon spice cookies, shortbread cookies  Some cookies are topped with orange and yellow',
 'Freshness Guaranteed Sugared Shortbread Cookies, 16 oz, 18 Count  Shortbread cookies topped with a sugared coating  Nut-Free',
 'Harvest themed frosted sugar cookies  Sugar cookies with yellow and orange frosting  Topped with festive and multicolor

#### Extracting Weight

In [27]:
def extract_weight(description):
    # Use a regex to find weight in the format 'X OZ'
    if description:
        match = re.search(r'(\d+\.?\d*)\s*OZ', description, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return None

In [28]:
weights = [extract_weight(i) for i in new_desc]

In [29]:
weights[:10]

[None, None, None, 13.5, None, 16.0, None, None, None, None]

#### Extracting Rating

In [30]:
rating = [i.get('rating', {}).get('averageRating', None) for i in fall_items]

In [31]:
rating[:10]

[0, 0, 0, 0, 0, 0, 0, 0, None, 0]

#### Extracting Number of Reviews

In [32]:
reviews = [i.get('rating', {}).get('numberOfReviews', None) for i in fall_items]

In [33]:
reviews[:10]

[0, 0, 0, 0, 0, 0, 0, 0, None, 0]

Let's validate the data to make sure all information was obtained and all lists are the same size.

In [34]:
print(len(name))
print(len(new_prices))
print(len(new_desc))
print(len(weights))
print(len(rating))
print(len(reviews))

37
37
37
37
37
37


All lists contain 37 items. Now that all data has been obtained, the information will be joined into a DataFrame.

In [35]:
import pandas as pd

In [36]:
bakery_data = pd.DataFrame()

In [37]:
bakery_data['Title'] = name
bakery_data['Price'] = new_prices
bakery_data['Description'] = new_desc
bakery_data['Weight'] = weights
bakery_data['Rating'] = rating
bakery_data['NumReviews'] = reviews

In [38]:
bakery_data.head(10)

,Title,Price,Description,Weight,Rating,NumReviews
0,Freshness Guaranteed Tri Pumpkin Face Sugar Co...,4.48,Freshness Guaranteed Tri Pumpkin Face Sugar Co...,NaN,0.0,0.0
1,Freshness Guaranteed Harvest Orange Frosted Su...,3.98,Sugar cookies topped with creamy orange frosti...,NaN,0.0,0.0
2,"Freshness Guaranteed Pumpkin Cake Roll, 18 oz,...",5.84,"Pumpkin Cake Roll, Kosher certified, contains ...",NaN,0.0,0.0
3,Freshness Guaranteed Pumpkin Shaped Vanilla Fr...,4.48,Freshness Guaranteed Pumpkin Shaped Vanilla Fr...,13.5,0.0,0.0
4,"Freshness Guaranteed Assorted Harvest Cookies,...",6.47,Includes about 45 cookies Flavors include cho...,NaN,0.0,0.0
5,Freshness Guaranteed Sugared Shortbread Cookie...,7.47,Freshness Guaranteed Sugared Shortbread Cookie...,16.0,0.0,0.0
6,"Freshness Guaranteed Frosted Sugar Cookies, Ha...",5.84,Harvest themed frosted sugar cookies Sugar co...,NaN,0.0,0.0
7,Walmart Freshness Guaranteed Fall Vanilla & Ch...,3.98,Contains 12 bite-sized cupcakes Assortment of...,NaN,0.0,0.0
8,None,NaN,None,NaN,NaN,NaN
9,Freshness Guaranteed Half Moon Iced Fall Baker...,3.98,Freshness Guaranteed Half Moon Iced Fall Baker...,NaN,0.0,0.0


The dataframe contains all necessary information. When references with [Walmart's fall bakery collection](https://www.walmart.com/browse/food/seasonal-bakery/976759_976779_7443156_4622028?povid=976759_ItemCarousel_4302028_Fallbakerytreats_ViewAll_Rweb_Sept_05) the information is correct and matches up to the appropriate item.

Now this data will be cleaned.

### Cleaning Data

In [39]:
bakery_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        36 non-null     object 
 1   Price        35 non-null     float64
 2   Description  35 non-null     object 
 3   Weight       12 non-null     float64
 4   Rating       36 non-null     float64
 5   NumReviews   36 non-null     float64
dtypes: float64(4), object(2)
memory usage: 1.9+ KB


In [40]:
# Where number of reviews is Nan, replace with 0
bakery_data['NumReviews'] = bakery_data['NumReviews'].fillna(0).astype(int)

In [41]:
bakery_data['NumReviews'][:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: NumReviews, dtype: int32

In [42]:
bakery_data.loc[bakery_data['NumReviews'] == 0, 'Rating'] = None

In [43]:
# Drop items where the price or title is None
bakery_data.dropna(subset=['Title', 'Price'], inplace=True)

In [44]:
bakery_data

,Title,Price,Description,Weight,Rating,NumReviews
0,Freshness Guaranteed Tri Pumpkin Face Sugar Co...,4.48,Freshness Guaranteed Tri Pumpkin Face Sugar Co...,NaN,NaN,0
1,Freshness Guaranteed Harvest Orange Frosted Su...,3.98,Sugar cookies topped with creamy orange frosti...,NaN,NaN,0
2,"Freshness Guaranteed Pumpkin Cake Roll, 18 oz,...",5.84,"Pumpkin Cake Roll, Kosher certified, contains ...",NaN,NaN,0
3,Freshness Guaranteed Pumpkin Shaped Vanilla Fr...,4.48,Freshness Guaranteed Pumpkin Shaped Vanilla Fr...,13.5,NaN,0
4,"Freshness Guaranteed Assorted Harvest Cookies,...",6.47,Includes about 45 cookies Flavors include cho...,NaN,NaN,0
5,Freshness Guaranteed Sugared Shortbread Cookie...,7.47,Freshness Guaranteed Sugared Shortbread Cookie...,16.0,NaN,0
6,"Freshness Guaranteed Frosted Sugar Cookies, Ha...",5.84,Harvest themed frosted sugar cookies Sugar co...,NaN,NaN,0
7,Walmart Freshness Guaranteed Fall Vanilla & Ch...,3.98,Contains 12 bite-sized cupcakes Assortment of...,NaN,NaN,0
9,Freshness Guaranteed Half Moon Iced Fall Baker...,3.98,Freshness Guaranteed Half Moon Iced Fall Baker...,NaN,NaN,0
10,Freshness Guaranteed Assorted Summer Bakery Co...,9.94,Freshness Guaranteed Assorted Cookies,NaN,NaN,0


Now that the data is in an acceptable format. We are ready to load the data into the Snowflake database.

In [45]:
# Checking longest description to determine needed length for storage
filtered_desc = [desc for desc in new_desc if desc is not None]
len(max(filtered_desc, key=len))

192

In [52]:
# Checking longest title to determine needed length for storage
filtered_title = [i for i in name if i is not None]
len(max(filtered_title, key=len))

184

In the creation of the bakery_tems table, description will be VARCHAR(200) and title will be VARCHAR(200).

In [47]:
# Rename columns before insertion:
bakery_data.rename(columns=lambda x: x.upper(), inplace=True)
bakery_data.rename(columns={'NUMREVIEWS': 'NUM_REVIEWS'}, inplace=True)

In [48]:
bakery_data.columns

Index(['TITLE', 'PRICE', 'DESCRIPTION', 'WEIGHT', 'RATING', 'NUM_REVIEWS'], dtype='object')

### Loading Data

In [49]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import os

# Retrieve connection info from environment variables
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
database = os.getenv('SNOWFLAKE_DATABASE')
schema = os.getenv('SNOWFLAKE_SCHEMA')

# Establish connection
try:
    conn = snowflake.connector.connect(
        user=user,
        password=password,
        account=account,
        warehouse=warehouse,
        database=database,
        schema=schema,
        role='ACCOUNTADMIN'
    )

    if conn:
        cur = conn.cursor()
        
        write_pandas(conn, bakery_data, 'BAKERY_ITEMS')
        print("Data successfully loaded.")

    cur.close()
    conn.close()
   
except Exception as e:
    print(f"Error: {e}")

Data successfully loaded.
